In [1]:
import pandas as pd
null="\u03bb"

In [195]:

class FiniteAutomaton(object):
    '''
    tran_table: pandas dataframe, to keep transition table
    '''
    def __init__(self, csv_file):
        '''
        Constructor
        Input: csv file of following format
        
        states,input_a,input_b,isInitial,isFinal
        1,2,2,TRUE,TRUE
        2,3,4,FALSE,TRUE
        3,,,FALSE,TRUE
        4,2,,FALSE,FALSE
        
        '''
        self.tran_table = pd.read_csv(csv_file, index_col="states", dtype="str") 
        self.tran_table.index = self.tran_table.index.map(str)
        self.tran_table.is_initial = self.tran_table["is_initial"].map({'FALSE':False, 'TRUE':True})
        self.tran_table.is_final = self.tran_table["is_final"].map({'FALSE':False, 'TRUE':True}) 

        
        # find final states, intial state  and symbols 
        self.q0= self.tran_table.loc[self.tran_table["is_initial"]==True].index.values[0]
        
        self.final_states=set(self.tran_table.loc[self.tran_table["is_final"]==True].index.values.tolist())
        
        self.symbols= [x for x in list(self.tran_table.columns) if x.find("input")!=-1]
        
        self.states= self.tran_table.index.values.tolist()
        
        
    def __str__(self):
        string= "inital Stated="+str(self.q0)+"\n"        
        string= string+"final states: "+ str(self.final_states)+"\n"
        string= string+"symbols: "+ str(self.symbols)+"\n"
        string= string+"states: "+str(self.states)+"\n"
        string= string+"Transition table\n"+str(self.tran_table)
        return string
    def print_trans_table(self):
        print("Transition table\n"+str(self.tran_table))
        
    def pair_table(self):
                # create table to store transitions between each pair of state
        table= pd.DataFrame(index=self.states+["NI","NF"], columns=self.states+["NI","NF"])

        # add tran(NI,null)=q0 and tran(finalstates, null)=NF
        table.loc["NI",self.q0]= null
        table.loc[self.final_states,"NF"]= null


        # add the initial transitions from given FA to table
        for x in self.states: # sourse
            for s in self.symbols: # symbol
                dest=self.tran_table.loc[x, s]
                if not pd.isnull(dest):  #destination if not nan
                    dest= dest.split(",")
                    for d in dest:
                        old_value=table.loc[x,d]
                        current_value= s.replace("input_","").replace("null", null)
                        if  not pd.isnull(old_value):
                            #initially table location is not nan
                            # add this transition to prevous transition (union) with brackets
                            table.loc[x,d]=   old_value+ " + "+current_value
                        else:
                            table.loc[x,d]= current_value          
        return table
        

        
    def to_RE(self, verbose=False):
        table=self.pair_table()
        if verbose:
            print(table)
        # eliminate states one by one
        # states will be elimiated in order of their occurance in DFA index
        for x in self.states:
            
            inward=[i for i in table.loc[~table[x].isna(), x].index.values if i!=x]            

            outward= [y for y in table.loc[~table.loc[x].isna()].index.values if y!=x]
            loop= table.loc[x,x]  if table.loc[x,x]==table.loc[x,x] else "" 
            print(loop)
            loop= "("+loop+")*" if len(loop)>1 else loop+"*" if len(loop)==1 else ""
           
            if verbose:
                print("Eliminating state ",x)
                print("inward=", inward)   
                print("outward= ",outward )
                print("loop", loop)

            for i in inward:
                for o in outward:

                    
                    REin= (table.loc[i,x] if table.loc[i,x]==table.loc[i,x] else "")
                    REout=(table.loc[x,o] if table.loc[x,o]==table.loc[x,o] else "")
                    REold=(table.loc[i,o] if table.loc[i,o]==table.loc[i,o] else "")
                    
                    if verbose:
                        print(i, "-->", x, REin, " ; ", 
                              ("loop "+loop if loop!="" else "no loop") , " ; ", 
                              x,"-->", o, REout,  " ; ",  
                              "old RE from " + i, " to "+ o+" "+REold if REold!="" else ""
                             )
                    
                    if (REin==null and REout+loop!=""):
                        REin=""
                    if (REout==null and REin+loop!=""):
                        REout=""
                    if (REin==null and REout==null):
                        REin=""
                    if ("+" in REin and loop+REout!=""):
                        REin="("+REin+")"
                    if ("+" in REout and loop+REin!=""):
                        REout="("+REout+")"
                    RE= REin + loop + REout 
                    if REold!="":
                        RE=REold+" + "+RE
                    table.loc[i,o]= RE
                        
                    
                    if verbose: 
                        print(i,"-->", o, "through", x, "is", RE, "as")

                    
            # delete row and column of that state        
            table = table.drop(x, 1)
            table = table.drop(x, 0)
            if verbose:
                print(table) 
        print("Regular expression\n", table.loc["NI", "NF"])
            

In [196]:
FA= FiniteAutomaton("FA CSV input files/DFA8.csv")
print(FA)
FA.pair_table()                                   

inital Stated=A
final states: {'B', 'A'}
symbols: ['input_a', 'input_b']
states: ['A', 'B', 'C']
Transition table
       input_a input_b  is_initial  is_final
states                                      
A            B       B        True      True
B            C       A       False      True
C            C       A       False     False


,A,B,C,NI,NF
A,NaN,a + b,NaN,NaN,λ
B,b,NaN,a,NaN,λ
C,b,NaN,a,NaN,NaN
NI,λ,NaN,NaN,NaN,NaN
NF,NaN,NaN,NaN,NaN,NaN


In [198]:
FA.to_RE()


b(a + b)
a + (b(a + b))(b(a + b))*a
Regular expression
 λ + (a + b)(b(a + b))*(λ + b) + ((a + b)(b(a + b))*a)(a + (b(a + b))(b(a + b))*a)*(b + (b(a + b))(b(a + b))*(λ + b))
